<a href="https://colab.research.google.com/github/evtaktasheva/hse-nlp/blob/main/NLP_HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Часть 0: сбор данных
Возьмем отзывы на музыкальные инструменты в качестве рабочего файла

In [1]:
!wget http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Musical_Instruments_5.json.gz  # сами отзывы
!wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_Musical_Instruments.json.gz  # метаданные

--2021-12-03 19:30:24--  http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Musical_Instruments_5.json.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2460495 (2.3M) [application/x-gzip]
Saving to: ‘reviews_Musical_Instruments_5.json.gz’

reviews_Musical_Ins 100%[===================>]   2.35M  3.64MB/s    in 0.6s    

2021-12-03 19:30:25 (3.64 MB/s) - ‘reviews_Musical_Instruments_5.json.gz’ saved [2460495/2460495]

--2021-12-03 19:30:25--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_Musical_Instruments.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 172497344 (165M) [application/octet-stream]
Saving to: ‘meta_Musical_Instruments.json.gz’



In [2]:
import pandas as pd
import gzip

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

In [3]:
df = getDF('reviews_Musical_Instruments_5.json.gz')
meta = getDF('meta_Musical_Instruments.json.gz')

In [4]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A2IBPI20UZIR0U,1384719342,"cassandra tu ""Yeah, well, that's just like, u...","[0, 0]","Not much to write about here, but it does exac...",5.0,good,1393545600,"02 28, 2014"
1,A14VAT5EAX3D9S,1384719342,Jake,"[13, 14]",The product does exactly as it should and is q...,5.0,Jake,1363392000,"03 16, 2013"
2,A195EZSQDW3E21,1384719342,"Rick Bennette ""Rick Bennette""","[1, 1]",The primary job of this device is to block the...,5.0,It Does The Job Well,1377648000,"08 28, 2013"
3,A2C00NNG1ZQQG2,1384719342,"RustyBill ""Sunday Rocker""","[0, 0]",Nice windscreen protects my MXL mic and preven...,5.0,GOOD WINDSCREEN FOR THE MONEY,1392336000,"02 14, 2014"
4,A94QU4C90B1AX,1384719342,SEAN MASLANKA,"[0, 0]",This pop filter is great. It looks and perform...,5.0,No more pops when I record my vocals.,1392940800,"02 21, 2014"


In [5]:
meta.head()

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes,details
0,"[Musical Instruments, Drums & Percussion, Hand...",,[Cricket Rubbing the spine with the wooden sti...,,Wooden Percussion 2 Piece Set of 3 Inch Cricke...,"[B00NP8GYVS, B00NP80XMO, B00NP8M098]",,WADSUWAN SHOP,"[Wood percussion, Owl whistle*, Includes woode...","[>#141,729 in Musical Instruments (See Top 100...",[],Musical Instruments,,"December 2, 2013",,0000989983,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
1,"[Musical Instruments, Drums & Percussion, Hand...",,[Frog - Rubbing its spine with the wooden stic...,,"Wooden Percussion 3 Piece Set Frog, Cricket an...","[B00NP8GYVS, B00NP80XMO, B01MY48HK5, B00AZZ1AJ...",,WADSUWAN SHOP,"[Wood percussion, Small 3 inches, Creates orig...","[>#1,622 in Musical Instruments (See Top 100 i...",[],Musical Instruments,,"December 2, 2013",$0.91,0000098906,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
2,"[Musical Instruments, Instrument Accessories, ...",,[Vivaldi's famous set of four violin concertos...,,Hal Leonard Vivaldi Four Seasons for Piano (Or...,[],,Hal Leonard,"[., ., .]","[>#330,653 in Musical Instruments (See Top 100...",[],Musical Instruments,,"May 10, 2011",$62.93,0041291905,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
3,[],,"[The Turn of the Screw (op. 54) vocal score, p...",,The Turn of the Screw (vocal score),"[0486266842, 0793507669, 0393008789, 142341280...",,Boosey &amp; Hawkes,[],"[>#86,354 in Musical Instruments (See Top 100 ...",[],Musical Instruments,,"May 23, 2007",$107.79,0060015500,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
4,[],,[],,Suite for Organ (including the Trumpet Volunta...,[],,,[],"[>#482,025 in Musical Instruments (See Top 100...",[],Musical Instruments,,"February 8, 2013",,0193757710,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN


# Часть 1: придумываем методы решения

## Метод 1.1
1. Т.к. у нас есть вся мета информация, то мы можем выделить самые частотные слова в названии товаров/их категории. Таким образом мы скорее всего выделим названия инструментов, типа `гитара`, `синтезатор` и т.д., что избавит нас от ручного написания всех типов продуктов, что есть в датасете (это можно сделать просто каким-нибудь каунтером)
2. Теперь у нас есть базовые категории товаров, можем найти их упоминания в отдельных текстах, например собрать ±2 слова вокруг, получить 2 и 3-граммы с нужным словом и дальше пойти двумя путями:    
    - взять самые частотные сочетания из отзывов (но тогда получим скорее всего какие-то и рандомные штуки, например если какой-то товар очень плохой, то можем и принять bad guitar за название)
    - взять те сочетания, которые встречаются в названии товаров (этот способ кажется хорошо отфильтрует все лишнее, но отфильтрует и немного измененные названия)

## Метод 1.2
Если использовать только данные из отзывов, то взять просто ключевые слова, выделить их, например, через tfidf или любой другой метод доставания ключевых слов, потом отфильтровать по порогу частотности, оставить только существительные. минус: выделим только основные аспекты, но не упоминания с названиями товара (`гитара`, но не `гитара Fender`). Плюс: скорее всего выделим правда в основном только аспекты, а не рандомные слова.

## Метод 2
Похож на метод 1, но заменим пункт 1 на морфологические шаблоны: выделим все сочетания типа NOUN + NOUN и тд и применим к отзывам. Затем сможем отфильтровать по пути 2

## Метод 3
Аналогично методу 2.1, только брать не просто морфологические шаблоны, а подключить еще и NER, тогда брать все, что подходит под NOUN + ORG и тд.

Минусы обоих подходов: не факт, что все организвации и названия товаров будут размечаться как ORG или что-то подобное, что поможет их выделить, + мы не выделим просто упоминания товаров без названия (типа `гитара`,
`барабан`) 



# Часть 2: Пробуем закодить

Кажется, имеет смысл закодить первый способ, т.к. он, насколько я знакома с ABSA, самый популярный из базовых.

In [6]:
!pip install --quiet spacy
!spacy download en_core_web_sm

     |████████████████████████████████| 12.0 MB 4.7 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [7]:
from tqdm.auto import tqdm
tqdm.pandas()

In [8]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [9]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
STOPS = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [10]:
from collections import Counter


texts = df.reviewText.str.lower()
lemmatized = texts.progress_apply(lambda x: ' '.join([t.text for t in nlp(x) if t.pos_ != 'PUNCT']))

def extract_words(texts):
    counter = []
    for text in tqdm(texts):
        text = nlp(text)
        tokens = [t.text for t in text if t.text not in STOPS and t.pos_ == 'NOUN']
        counter.extend(tokens)
    return Counter(counter)

  0%|          | 0/10261 [00:00<?, ?it/s]

In [11]:
counter = extract_words(lemmatized)

  0%|          | 0/10261 [00:00<?, ?it/s]

In [12]:
counter.most_common(10)

[('guitar', 5219),
 ('strings', 3322),
 ('pedal', 2493),
 ('price', 2152),
 ('sound', 2132),
 ('amp', 1857),
 ('quality', 1757),
 ('time', 1528),
 ('tone', 1493),
 ('one', 1448)]

Как видим, мы действительно выделили что-то похожее на нужные нам вещи, но в них попадают и другие частые аспекты: цена и тд, что в настоящий момент нам не надо.  Давайте попробуем сделать то же самое с названиями из метаданых:

In [13]:
titles = meta.title.str.lower().to_list()
title_counter = extract_words(titles)

  0%|          | 0/120310 [00:00<?, ?it/s]

In [14]:
title_kw = title_counter.most_common(20)
title_kw

[('guitar', 20591),
 ('case', 7334),
 ('pack', 4451),
 ('bag', 4256),
 ('cable', 4219),
 ('microphone', 3829),
 ('bass', 3749),
 ('pedal', 3412),
 ('strings', 3332),
 ('size', 2980),
 ('string', 2861),
 ('violin', 2744),
 ('inch', 2725),
 ('music', 2663),
 ('stand', 2628),
 ('audio', 2538),
 ('light', 2440),
 ('strap', 2397),
 ('system', 2373),
 ('instrument', 2372)]

Это выглядит уже прям очень хорошо! Но мы не учитываем частотных сочетаний, наверное стоит достать частотные биграммы: воспользуемся методом из первой домашки)

In [15]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer


cv = CountVectorizer(max_df=0.85, stop_words=STOPS, max_features=10000, ngram_range=(2,2))
word_count_vector = cv.fit_transform(lemmatized.to_list())

tfidf_transformer = TfidfTransformer(smooth_idf=True, use_idf=True)
tfidf_transformer.fit(word_count_vector)

TfidfTransformer()

In [16]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

def extract_topn_from_vector(feature_names, sorted_items, tol=0.1):
    """get the feature names and tf-idf score of items"""
    
    sorted_items = [item for item in sorted_items if item[1] > tol]  

    score_vals = []
    feature_vals = []

    for idx, score in sorted_items:
        fname = feature_names[idx]
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])

    #create a tuples of feature,score
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

In [17]:
def tfidf_keywords(text, tol=0.1):
    feature_names = cv.get_feature_names_out()
    tf_idf_vector = tfidf_transformer.transform(cv.transform([text]))
    sorted_items = sort_coo(tf_idf_vector.tocoo())
    keywords = extract_topn_from_vector(feature_names, sorted_items, tol)

    return set(keywords.keys())

In [18]:
kw = tfidf_keywords(' '.join(meta.title.to_list()), tol=0.07)

In [19]:
kw

{'acoustic electric',
 'acoustic guitar',
 'audio technica',
 'bass guitar',
 'classical guitar',
 'condenser microphone',
 'drum set',
 'effects pedal',
 'electric bass',
 'electric guitar',
 'full size',
 'gig bag',
 'guitar bass',
 'guitar pick',
 'guitar picks',
 'guitar strap',
 'guitar strings',
 'instrument cable',
 'microphone cable',
 'music stand',
 'polishing cloth',
 'power supply',
 'seymour duncan',
 'snare drum',
 'vic firth',
 'xlr female'}

In [20]:
kw = list(kw) + [i[0] for i in title_kw]

In [21]:
kw

['guitar bass',
 'guitar pick',
 'polishing cloth',
 'drum set',
 'music stand',
 'microphone cable',
 'xlr female',
 'effects pedal',
 'audio technica',
 'power supply',
 'condenser microphone',
 'seymour duncan',
 'gig bag',
 'vic firth',
 'guitar picks',
 'classical guitar',
 'acoustic electric',
 'snare drum',
 'electric guitar',
 'guitar strings',
 'instrument cable',
 'full size',
 'guitar strap',
 'acoustic guitar',
 'bass guitar',
 'electric bass',
 'guitar',
 'case',
 'pack',
 'bag',
 'cable',
 'microphone',
 'bass',
 'pedal',
 'strings',
 'size',
 'string',
 'violin',
 'inch',
 'music',
 'stand',
 'audio',
 'light',
 'strap',
 'system',
 'instrument']

Теперь у нас есть все аспекты, но можем видим, что в них попадают не только товары, но и другие вещи типа размера и тд, можем их убрать ручками


In [23]:
kw.remove('size')
kw.remove('music')
kw.remove('inch')
kw.remove('full size')
kw.remove('light')

По идее нам хватит и такого, но хочется попробовать достать компании, чтобы понимать про какие именно инструменты что-то говорят, давайте попробуем достать все штуки, где несколько слов подряд идут с большой буквы - так выделим компании и названия товаров

In [24]:
import re

def get_names(text):
    text = ' '.join([t.text for t in nlp(text)])
    return set([i.strip() for i in re.findall('((?:[A-Z][a-z]*?(?:\s|\b)){2,3})', text) if 'I' not in i.split()])

Проверим:

In [25]:
df.reviewText[7]

"I now use this cable to run from the output of my pedal chain to the input of my Fender Amp. After I bought Monster Cable to hook up my pedal board I thought I would try another one and update my guitar. I had been using a high end Planet Waves cable that I bought in the 1980's... Once I found out the input jacks on the new Monster cable didn't fit into the Fender Strat jack I was a little disappointed... I didn't return it and as stated I use it for the output on the pedal board. Save your money... I went back to my Planet Waves Cable...I payed $30.00 back in the eighties for the Planet Waves which now comes in at around $50.00. What I'm getting at is you get what you pay for. I thought Waves was a lot of money back in the day...but I haven't bought a guitar cable since this one...20 plus years and still working...Planet Waves wins."

In [26]:
names = get_names(df.reviewText[7])
names

{'Fender Amp',
 'Fender Strat',
 'Monster Cable',
 'Planet Waves',
 'Planet Waves Cable'}

Ура  мы выделили что-то нормальное. Теперь у нас есть два пути: 
1. Отфильтровать все наши полученные названия и оставить только те, где есть указание на один из товаров из уже выделенных нами аспектов
2. Оставить все выделенные названия

Мне кажется, что можно оставить все названия, потому что часто в отзывах продукты могут упоминаться только по нахванию компании (см. пример выше, просто `Planet Waves` помимо `Planet Waves cable`, когда сравниваются разные кабели)

Если же мы отсортируем и оставим только те упоминания, где встречаются наши аспекты, выделенные заранее, то такие упоминания мы потеряем, зато наверное лучше отделим аспекты от рандомных упоминаний.. (С другой стороны каков шанс, что 2-3 слова подряд с большой буквы это что-то другое кроме как товара?) 

Но пойду по более safe пути и лучше найду меньше, но точнее:

In [27]:
def product_intersection(aspects, names):
    filtered_names = []
    for aspect in aspects:
        for name in names:
            if aspect in name.lower():
                filtered_names.append(' '.join([n.lower() for n in name.split() if n.lower() not in STOPS]))
    return filtered_names

Снова проверим

In [28]:
product_intersection(kw, names)

['planet waves cable', 'monster cable']

Как видим, мы отфильтровали `Fender Amp`, что вроде как тоже продукт, что не очень( Можно его, конечно, добавить ручками, но мы не сможем сделать так со всеми товарами, так что будем считать, что его не существует.. 

In [29]:
names = df.reviewText.progress_apply(lambda x: product_intersection(kw, get_names(x)))

  0%|          | 0/10261 [00:00<?, ?it/s]

In [30]:
filtered_names = []
for i in names.to_list():
    if i != []:
        filtered_names.extend(i)
filtered_names = set(filtered_names)

In [31]:
list(filtered_names)[:10]

['flexible microphone',
 'tenor guitar',
 'w guitar combo',
 'guitar instrument cable',
 'guitar combos',
 'ernie ball strings',
 'seymour duncan humbuckers',
 'fender standard telecaster',
 'dimebag darrels',
 'standard']

Выглядит ничего! Посмотрим на количество найденных слов:

In [32]:
len(filtered_names)

450

Соединим с нашими keywords:

In [33]:
new_kw = list(set(kw) | filtered_names)

In [34]:
new_kw[:10]

['mr scary guitars',
 'electric bass guitar',
 'flexible microphone',
 'tenor guitar',
 'drum set',
 'electric violin',
 'w guitar combo',
 'guitar instrument cable',
 'guitar combos',
 'ernie ball strings']

In [35]:
len(new_kw)

464

Ура у нас есть список упоминаний! Давайте соберем нграммы с ними. Но сначала преобразуем тексты для удобства:

In [126]:
gram2word_kw = {word: ''.join(word.split()) for word in new_kw}
word2gram_kw = {y: x for (x , y) in gram2word_kw.items()}

In [127]:
list(gram2word_kw.items())[:5]

[('mr scary guitars', 'mrscaryguitars'),
 ('electric bass guitar', 'electricbassguitar'),
 ('flexible microphone', 'flexiblemicrophone'),
 ('tenor guitar', 'tenorguitar'),
 ('drum set', 'drumset')]

In [128]:
list(word2gram_kw.items())[:5]

[('mrscaryguitars', 'mr scary guitars'),
 ('electricbassguitar', 'electric bass guitar'),
 ('flexiblemicrophone', 'flexible microphone'),
 ('tenorguitar', 'tenor guitar'),
 ('drumset', 'drum set')]

In [50]:
def change_text(text, aspect, asp2word):
    return re.sub(r'\b{}\b'.format(aspect), asp2word[aspect], text)
    text_parts = text.split(f' {aspect} ')
    return asp2word[aspect].join(text_parts)

In [54]:
change_text(lemmatized[7], 'planet waves cable', gram2word_kw)

"i now use this cable to run from the output of my pedal chain to the input of my fender amp after i bought monster cable to hook up my pedal board i thought i would try another one and update my guitar i had been using a high end planetwavescable that i bought in the 1980 's once i found out the input jacks on the new monster cable did n't fit into the fender strat jack i was a little disappointed i did n't return it and as stated i use it for the output on the pedal board save your money i went back to my planetwavescable i payed $ 30.00 back in the eighties for the planet waves which now comes in at around $ 50.00 what i 'm getting at is you get what you pay for i thought waves was a lot of money back in the day but i have n't bought a guitar cable since this one 20 plus years and still working planet waves wins"

In [58]:
def get_ngrams(text, aspect):
    ngrams = []
    text_parts = text.split(aspect)
    # print(text_parts)
    for i, part in enumerate(text_parts):
        # print('part:', part)
        if part == '' or part == ' ':
            continue
        tokens = part.split()
        if i == 0:
            # print(0)
            grams = tokens[(-2 if len(tokens) >=2 else -1):]
            ngrams.append(grams[-1] + f' {aspect}')
            ngrams.append(' '.join(grams) + f' {aspect}')
        elif i != len(text_parts)-1:
            # print(1)
            # left context
            grams = tokens[:(2 if len(tokens) >=2 else 1):]
            ngrams.append(f'{aspect} ' + grams[0])
            ngrams.append(f'{aspect} ' + ' '.join(grams))

            # right context
            grams = tokens[(-2 if len(tokens) >=2 else -1):]
            ngrams.append(grams[-1] + f' {aspect}')
            ngrams.append(' '.join(grams) + f' {aspect}')
        else:
            # print(2)
            grams = tokens[:(2 if len(tokens) >=2 else 1):]
            ngrams.append(f'{aspect} ' + grams[0])
            ngrams.append(f'{aspect} ' + ' '.join(grams))
    return ngrams

Проверим:

In [59]:
lemmatized[7]

"i now use this cable to run from the output of my pedal chain to the input of my fender amp after i bought monster cable to hook up my pedal board i thought i would try another one and update my guitar i had been using a high end planet waves cable that i bought in the 1980 's once i found out the input jacks on the new monster cable did n't fit into the fender strat jack i was a little disappointed i did n't return it and as stated i use it for the output on the pedal board save your money i went back to my planet waves cable i payed $ 30.00 back in the eighties for the planet waves which now comes in at around $ 50.00 what i 'm getting at is you get what you pay for i thought waves was a lot of money back in the day but i have n't bought a guitar cable since this one 20 plus years and still working planet waves wins"

In [60]:
get_ngrams(lemmatized[7], 'planet waves cable')

['end planet waves cable',
 'high end planet waves cable',
 'planet waves cable that',
 'planet waves cable that i',
 'my planet waves cable',
 'to my planet waves cable',
 'planet waves cable i',
 'planet waves cable i payed']

In [61]:
get_ngrams(lemmatized[7], 'guitar')

['my guitar',
 'update my guitar',
 'guitar i',
 'guitar i had',
 'a guitar',
 'bought a guitar',
 'guitar cable',
 'guitar cable since']

Отлично! Теперь можем достать все нграммы из всех текстов

In [64]:
def process_text(text, aspects, asp2word):
    for aspect in aspects:
        text = change_text(text, aspect, gram2word_kw)
    return text

In [66]:
one_worded_texts = lemmatized.progress_apply(process_text, aspects=new_kw, asp2word=gram2word_kw)

  0%|          | 0/10261 [00:00<?, ?it/s]

In [67]:
def extract_ngrams(texts, aspects):
    ngrams = []
    progress_bar = tqdm(total=len(texts)*len(aspects))
    for i, text in enumerate(texts):
        for aspect in aspects:
            if gram2word_kw[aspect] in text:
                grams = get_ngrams(text, gram2word_kw[aspect])
                ngrams.extend(grams)
            progress_bar.update(1)
    progress_bar.close()
    return ngrams

In [68]:
grams = extract_ngrams(one_worded_texts, new_kw)

  0%|          | 0/4761104 [00:00<?, ?it/s]

In [69]:
grams[:10]

['mike stand',
 'the mike stand',
 'stand secure',
 'stand secure enough',
 'connectors bass',
 'gold connectors bass',
 'bass sounds',
 'bass sounds great',
 'used monstercable',
 'have used monstercable']

In [70]:
len(grams)

147364

### Часть 3: метрики
Теперь можем достать самые частотные нграммы и отранжировать их

In [73]:
from nltk.collocations import *

In [75]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
trigram_measures = nltk.collocations.TrigramAssocMeasures()

In [314]:
bigram_finder = BigramCollocationFinder.from_documents(one_worded_texts.apply(str.split))
trigram_finder = TrigramCollocationFinder.from_documents(one_worded_texts.apply(str.split))
# отфильтруем по количеству встречаемости
bigram_finder.apply_freq_filter(3)
trigram_finder.apply_freq_filter(3)

#### PMI

In [315]:
bigrams_pmi = bigram_finder.score_ngrams(bigram_measures.pmi)
trigrams_pmi = trigram_finder.score_ngrams(trigram_measures.pmi)

Напишем вспомогательную функцию для фильтрации.

Спойлер, там много ранломных слов-характеристик гитар, поэтому попробуем почистить еще и на цифры и пунктцуацию + проверим на часть речи для однословных аспектов


In [316]:
def check_pos(gram, aspects):
    tokens = gram.split()
    for i, t in enumerate(nlp(gram)):
        if t.text in aspects:
            if aspects[t.text] == t.text and t.pos == 'NOUN':
                return gram
            else:
                tokens[i] = aspects[t.text]
            return ' '.join(tokens)
    return False


def filter_grams(collocations, word2asp):
    filtered = []
    for i, collocation in enumerate(collocations):
        col = []
        aspect = False
        alpha = True
        for j, word in enumerate(collocation[0]):
            if not word.isalpha():
                alpha = False
            elif word in word2asp:
                aspect = True
            col.append(word)
        col = ' '.join(col)
        if aspect and alpha:
            checked = check_pos(col, word2asp)
            if checked:
                filtered.append((checked, collocation[1]))
    return filtered

In [317]:
filtered_bigrams = filter_grams(bigrams_pmi, word2gram_kw)
filtered_bigrams[:10]

[('lap steel guitar', 14.660289702802492),
 ('pro winder string winder', 13.256774073418867),
 ('m audio fast track', 12.686761914163682),
 ('tubular guitar stand', 12.589900374911096),
 ('bronze acoustic guitar strings', 12.333124959548984),
 ('tabletop mic stand', 12.302098063928359),
 ('slinky string set', 12.241977071490787),
 ('teach classical guitar', 11.83431910257754),
 ('adjustable microphone boom', 11.340987077321955),
 ('strap lock systems', 11.294444491384924)]

Ура выглядит хорошо! Проделаем то же самое для триграмм

In [318]:
filtered_trigrams = filter_grams(trigrams_pmi, word2gram_kw)
filtered_trigrams[:10]

[('tb adjustable microphone boom', 28.87574589804059),
 ('phosphor bronze acoustic guitar strings', 26.24729736981574),
 ('carbon fiber guitar picks', 25.539696694908564),
 ('dual design straplok system', 24.856510658099296),
 ('tripod boom microphone stand', 23.821590127993677),
 ('mi audio crunch', 23.679004715505247),
 ('daisy chain power supply', 21.74285409116392),
 ('pedal board real estate', 21.156149481757538),
 ('young guitar students', 20.80042252889421),
 ('instrument clip on chromatic', 20.668659546955546)]

#### T-score

In [319]:
bigrams_t = bigram_finder.score_ngrams(bigram_measures.student_t)
trigrams_t = trigram_finder.score_ngrams(trigram_measures.student_t)

In [320]:
filtered_bigrams = filter_grams(bigrams_t, word2gram_kw)
filtered_bigrams[:10]

[('this pedal', 25.990897256788507),
 ('the guitar', 23.475056174957828),
 ('my guitar', 22.20969285657442),
 ('these strings', 20.651455673530837),
 ('your guitar', 19.286466482445096),
 ('the strings', 18.269583778055985),
 ('my guitars', 15.32587494477202),
 ('the strap', 13.42787412798336),
 ('strings are', 13.339778501790741),
 ('this stand', 13.319164958316046)]

In [321]:
filtered_trigrams = filter_grams(trigrams_t, word2gram_kw)
filtered_trigrams[:10]

[('of the guitar', 11.600245156359737),
 ('this pedal is', 11.536650894185016),
 ('on the guitar', 10.063423404407256),
 ('of my guitars', 8.81256633498553),
 ('on my guitar', 8.786368322589437),
 ('these strings are', 8.708129363478381),
 ('with this pedal', 7.4461958040451135),
 ('set of strings', 7.273999318774714),
 ('on the strings', 7.1671509903186035),
 ('the guitar and', 7.113562143599684)]

#### CHI

In [322]:
bigrams_chi = bigram_finder.score_ngrams(bigram_measures.chi_sq)
trigrams_chi = trigram_finder.score_ngrams(trigram_measures.chi_sq)

In [323]:
filtered_bigrams = filter_grams(bigrams_chi, word2gram_kw)
filtered_bigrams[:10]

[('lap steel guitar', 77675.53637530934),
 ('strap locks', 44825.76471563754),
 ('bronze acoustic guitar strings', 30950.119863961834),
 ('pro winder string winder', 29360.59792895801),
 ('tubular guitar stand', 24654.727351118945),
 ('teach classical guitar', 21899.67762113828),
 ('m audio fast track', 19776.64582405132),
 ('this pedal', 17439.631129631747),
 ('industry standard', 16216.963894414754),
 ('tabletop mic stand', 15146.633000326743)]

In [324]:
filtered_bigrams = filter_grams(trigrams_chi, word2gram_kw)
filtered_bigrams[:10]

[('tb adjustable microphone boom', 1970269585.0449219),
 ('phosphor bronze acoustic guitar strings', 318933367.3908473),
 ('carbon fiber guitar picks', 146396650.14731613),
 ('dual design straplok system', 121511797.39825328),
 ('tripod boom microphone stand', 44478088.67961312),
 ('mi audio crunch', 40293127.49803647),
 ('daisy chain power supply', 10649229.24498211),
 ('young guitar students', 9131057.595971018),
 ('pedal board real estate', 7030159.66590833),
 ('instrument clip on chromatic', 5000435.500810749)]

Лучше всего себя показал PMI, так что посмотрим на него получше. Объединим все полученные нграммы по нашим изначально вынутым аспектам (а не конкретным кпоминаниям продуктов)

In [325]:
filtered_bigrams = filter_grams(bigram_finder.score_ngrams(bigram_measures.pmi), word2gram_kw)
filtered_trigrams = filter_grams(trigram_finder.score_ngrams(trigram_measures.pmi), word2gram_kw)

In [367]:
def sort_freq(categories):
    for category, grams in categories.items():
        categories[category] = sorted(grams, key=lambda x: x[1], reverse=True)
    return categories
    
def combine_grams(ngrams, aspects, word2asp):
    categories = {}
    aspects = sorted(aspects, key=lambda x: len(x.split()), reverse=True)
    for ngram in ngrams:
        for aspect in aspects:
            if aspect in ngram[0]:
                if aspect not in categories:
                    categories[aspect] = []
                categories[aspect].append(ngram)
    return sort_freq(categories)

In [368]:
categories = combine_grams(filtered_bigrams+filtered_trigrams, kw, word2gram_kw)

Посмотрим на 5 категорий:

In [371]:
categories['guitar'][:10]

[('phosphor bronze acoustic guitar strings', 26.24729736981574),
 ('carbon fiber guitar picks', 25.539696694908564),
 ('young guitar students', 20.80042252889421),
 ('pawn shop guitar', 20.037160191019776),
 ('your typical guitar pick', 19.43378818633741),
 ('typical guitar pick are', 18.46831095122346),
 ('les paul electric guitar', 18.21280033506963),
 ('laguna acoustic guitar they', 17.89408504552862),
 ('guitar pick are bigger', 17.455710914443824),
 ('studio album guitar', 17.45219769029863)]

In [372]:
categories['pedal'][:10]

[('pedal board real estate', 21.156149481757538),
 ('most pedal boards', 15.64611179550218),
 ('an expression pedal', 15.39324835965946),
 ('analog delay pedal', 14.953668676117822),
 ('effects pedal works great', 14.410955375797915),
 ('the voodoo lab pedal power', 14.278552938610332),
 ('volume wah pedal', 13.916823315838485),
 ('clean boost pedal', 13.020761136110732),
 ('gain distortion pedal', 12.938577545831269),
 ('an eq pedal', 11.91846877668872)]

In [373]:
categories['guitar strap'][:10]

[('very nice guitar strap', 11.999642667200998),
 ('these guitar straps', 9.323676324954743),
 ('about a guitar strap', 8.818081693461366),
 ('my guitar straps', 8.220030813337715),
 ('this guitar strap is', 7.528782897627522),
 ('guitar strap button', 6.755148889143932),
 ('guitar strap but i', 6.746418907537446),
 ('guitar straps and', 6.697598110989382),
 ('on the guitar strap', 5.99814986635274),
 ('a guitar strap it', 5.287300230891276)]

In [374]:
categories['cable'][:10]

[('an xlr cable', 13.580932623142303),
 ('higher quality cable', 12.975660650208443),
 ('high quality cable', 11.267172403293806),
 ('cable works fine', 11.210727107994472),
 ('cable to connect', 11.178908358277262),
 ('more expensive cable', 11.031079308911124),
 ('usb cable but', 11.004306792923668),
 ('cable itself is', 11.003505941563727),
 ('well made cable', 10.54371012135175),
 ('cable appears to', 10.529094713206131)]

In [375]:
categories['microphone'][:10]

[('tb adjustable microphone boom', 28.87574589804059),
 ('tripod boom microphone stand', 23.821590127993677),
 ('a mono microphone', 12.275393655761121),
 ('built in microphone', 11.958534857636021),
 ('all around microphone', 11.840508382902744),
 ('a dynamic microphone', 11.700699490493786),
 ('on microphone stands', 11.343173154013364),
 ('adjustable microphone boom', 11.340987077321955),
 ('boom microphone stand', 10.345724996328615),
 ('microphone works well', 10.149728803410035)]

Как видим, все работает не очень хорошо( Однако я бы связада это с особенностью отзывов: в них все же чаще описываются отдельные аспекты продукта (цена, качество и т.д.), а не товар в целом, поэтому упоминаний товаров в них мало => такой метод выделения по частотным словам из названий продуктов не очень работает. Дальше еще немного недостатков:

+ конечно, способ выделения категорий, который я использую, не идеален, и можно было бы подумать что-то более sophisticated и точное, однако я решила, что для наших игрушечных примеров такой вариант подойдет

+ корпус довольно маленький, что стоит учитывать при подсчете статистик: большинство продуктов встречаются только один раз, что, конечно, сильно влияет на частотность тех или иных терминов

+ нужно подумать над фильтрацией, сейчас у нас есть две основные проблемы:    
    + из-за того, что в английском языке к нас много grammatical ambiguity, в частности, `guitar` может быть как существительным так и прилагательным, поэтому при выделении аспектов тяжело их фильтровать и даже частеречная разметка не всегда помогает
    + сейчас в нграммы попадают много неописательных вещей, хорошо бы подумать, как их отфильтровать: например, по частеречной разметке (оставить только прилагательные и конструкции типа PRON + VERB + NE (ср. `I love guitar`, но что делать с артиклями? другими вещями которые попадаются между)/ NE + VERB + ADV (ср. `quitar works well`) и др. но придумывать такие шаблоны долго и не всегда получится все учесть


Однако, видим и приятные вещи: например, мы видим и нужные нам сочетания, как `microphone works well`, `high quality cable`, `cable works fine`, `very nice guitar strap`, `effects pedal works great` и т.д.


В целом, если исправить все перечисленные выше недочеты, то, кажется, метод сработает. При этом так мы скорее узнаем general мнения об ассортименте магазина, но если мы захотим посмотреть про конкретные товары, то можно будет перед подачей в выделитель коллокаций отфильтровать корпус по нужным товарам.